In [1]:
import numpy as np
from scipy.stats import norm

In [6]:
class SortTree:
  def __init__(self, value):
    self.left = None
    self.value = value
    self.right = None
  def insert_val(self, _value):
    if _value < self.value:
      if self.left is None:
        self.left = SortTree(_value)
      else:
        self.left.insert_val(_value)
    else:
      if self.right is None:
        self.right = SortTree(_value)
      else:
        self.right.insert_val(_value)

def display_sort(_node):
  return list(filter(None, [i for b in [display_sort(_node.left) if isinstance(_node.left, SortTree) 
              else [getattr(_node.left, 'value', None)], [_node.value], display_sort(_node.right) 
              if isinstance(_node.right, SortTree) else [getattr(_node.right, 'value', None)]] for i in b]))

tree = SortTree(4)
for i in [5, 3, 1, 2, 8, 7, 4]:
  tree.insert_val(i)
  print(display_sort(tree))



[4, 5]
[3, 4, 5]
[1, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 8]
[1, 2, 3, 4, 5, 7, 8]
[1, 2, 3, 4, 4, 5, 7, 8]


In [7]:
class ArgSortTree:
  def __init__(self, value, index):
    self.left = [None,None]
    self.right = [None,None]
    self.value_index = [value,index]
  def insert_val(self, _value, _index):
    if _value < self.value_index[0]:
      if self.left ==[None,None]:
        self.left = ArgSortTree(_value,_index)
      else:
        self.left.insert_val(_value, _index)
    else:
      if self.right == [None,None]:
        self.right = ArgSortTree(_value,_index)
      else:
        self.right.insert_val(_value,_index)

def display_argsort(_node):
  return list(filter(None, [i for b in [display_argsort(_node.left) if isinstance(_node.left, ArgSortTree) 
            else [getattr(_node.left, 'value', None)], [_node.value_index], display_argsort(_node.right) 
            if isinstance(_node.right, ArgSortTree) else [getattr(_node.right, 'value', None)]] for i in b]))

j=0
tree = ArgSortTree(4,j)
for i in [5, 3, 1, 2, 8, 7, 4]:
  j = j+1
  tree.insert_val(i,j)
  sorted_index = display_argsort(tree)
  pos = [a[1] for a in sorted_index]
  rank = [0]*(j+1)
  for k in range(len(pos)):
    rank[pos[k]]= k
  print(rank)

[0, 1]
[1, 2, 0]
[2, 3, 1, 0]
[3, 4, 2, 0, 1]
[3, 4, 2, 0, 1, 5]
[3, 4, 2, 0, 1, 6, 5]
[3, 5, 2, 0, 1, 7, 6, 4]


In [48]:
class OnlineSort:
  def __init__(self, value, temporal_index,day_of_week):
    self.value = value
    self.temporal_index = temporal_index
    self.day_of_week = day_of_week

def is_larger(_OnlineSort1,_OnlineSort2):
  if _OnlineSort1.value>_OnlineSort2.value:
    return True
  else:
    return False

def initialize_data(raw_data): # data of at least first two weeks
  online_list = []
  online_eps = []
  sum_ = [0,0,0,0,0,0,0]
  day_ = [0,0,0,0,0,0,0]
  y_list = []
  starting_mean = np.mean(raw_data)
  for i in range(len(raw_data)):
    if i<4:
      y = raw_data[i] - starting_mean
    else:
      y = raw_data[i]-0.4*raw_data[i-1]-0.3*raw_data[i-2]-0.2*raw_data[i-3]-0.1*raw_data[i-4]
    y_list.append(y)
    item = OnlineSort(raw_data[i],i,i%7)  ## later consider missing days
    online_list.append(item)
    sum_[i%7] = sum_[i%7] + y
    day_[i%7] = day_[i%7] + 1
  for i in range(len(raw_data)):
    item = OnlineSort(y_list[i]-sum_[i%7]/day_[i%7], i, i%7)
    online_eps.append(item)
  new_data = {"data":online_list,"sum":sum_,"days":day_}
  return {"new_data":new_data,"online_eps":online_eps}
  
def gather_data(old_data,new_obs): ## in temporal order (after 2 weeks)
  n = len(old_data['data'])
  mu = 0.4*old_data['data'][n-1].value+0.3*old_data['data'][n-2].value+0.2*old_data['data'][n-3].value+0.1*old_data['data'][n-4].value
  new_y = new_obs.value - mu
  new_sum = old_data['sum'][new_obs.day_of_week] + new_y
  new_s = new_sum/(old_data['days'][new_obs.day_of_week]+1)
  delta = new_s - old_data['sum'][new_obs.day_of_week]/old_data['days'][new_obs.day_of_week]
  new_eps = OnlineSort(new_y - new_s,new_obs.temporal_index,new_obs.day_of_week)
  new_data = old_data
  new_data['data'] = old_data['data'].append(new_obs)
  new_data['sum'][new_obs.day_of_week] = new_sum
  new_data['days'][new_obs.day_of_week] = new_data['days'][new_obs.day_of_week]+1
  return {"delta":delta,"new_eps":new_eps,"new_data":new_data}
  
def argsort_OL(online_eps):  ## used once in the beginning but not online part
  value_list = []
  for i in range(len(online_eps)):
    value_list.append(online_eps[i].value)
  rank = np.argsort(np.array(value_list))
  sorted_eps = []
  for i in range(len(online_eps)):
    sorted_eps.append(online_eps[rank[i]])
  return sorted_eps

def split_update_eps(sorted_eps,new_eps,delta):
  day_eps = []
  rest_eps = []
  for i in range(len(sorted_eps)):
    if sorted_eps[i].day_of_week == new_eps.day_of_week:
      sorted_eps[i].value = sorted_eps[i].value - delta
      day_eps.append(sorted_eps[i])
    else:
      rest_eps.append(sorted_eps[i])
  ## insert new_eps in day_eps
  old_len = len(day_eps)
  for i in range(len(day_eps)):
    if is_larger(day_eps[i],new_eps):
      day_eps.insert(i,new_eps)
      break
  if len(day_eps)==old_len:
    day_eps.insert(old_len,new_eps)
  ## merge day_eps into rest_eps
  new_sorted_eps = [OnlineSort(0,0,0)]*(len(day_eps)+len(rest_eps))
  j = 0
  final_index = 0
  new_rank = -1
  for i in range(len(day_eps)):
    while(is_larger(day_eps[i],rest_eps[j])):
      new_sorted_eps[final_index] = rest_eps[j]
      j = j + 1
      final_index = final_index + 1
    new_sorted_eps[final_index] = day_eps[i]
    if day_eps[i].temporal_index == new_eps.temporal_index:
      new_rank = final_index
    final_index = final_index + 1
  if len(new_sorted_eps)>final_index:
    for i in range(len(new_sorted_eps)-final_index):
      new_sorted_eps[final_index+i]=rest_eps[j]
      j = j + 1
  return {"new_sorted_eps":new_sorted_eps,"rank":new_rank}

In [23]:
## this is a test case
raw_data = np.random.normal(size=23)
processed_data = initialize_data(raw_data)
old_data = processed_data["new_data"]
online_eps = processed_data["online_eps"]

In [24]:
old_data

{'data': [<__main__.OnlineSort at 0x21b5db65908>,
 'sum': [0.414876349722905,
  -1.236706146527491,
  -1.1379073695297155,
  0.6060059226140014,
  1.022218379720847,
  -2.4772202602335707,
  3.0884210767442593],
 'days': [4, 4, 3, 3, 3, 3, 3]}

In [25]:
sorted_eps = argsort_OL(online_eps)
sorted_eps

In [27]:
new_obs = OnlineSort(1.2,23,2)
new_info = gather_data(old_data,new_obs)
delta = new_info["delta"]
new_eps = new_info["new_eps"]
new_data = new_info["new_data"]

In [49]:
outcome = split_update_eps(sorted_eps,new_eps,delta)
new_sorted_eps = outcome["new_sorted_eps"]
new_rank = outcome["rank"]

In [50]:
new_rank

20

In [51]:
len(new_sorted_eps)

24

In [5]:
## variance update
## first apply split_update_eps on every feature, save all the ranks in one vector [ranks]
def cov2corr(cov):
  cov = np.asanyarray(cov)
  std_ = np.sqrt(np.diag(cov))
  corr = cov / np.outer(std_, std_)
  return corr

def var_update(ranks,n,old_var):  ## n means: this is the nth value since the very beginning 
  eps_tilde =  norm.ppf(ranks/(n+1))
  new_var = cov2corr((n-1)/n*old_var+1/n*np.outer(eps_tilde,eps_tilde))
  return new_var

def test_stat(ranks,n,new_var):
  eps_tilde =  norm.ppf(ranks/(n+1))
  stat = eps_tilde.dot(new_var).dot(eps_tilde)
  return stat

In [ ]:
## combine everything
## assume we have a dictionary to hold all the raw data {"feature1":[],"feature2":[],...}
##                                     all the sorted eps
## now we observe a new input [a,b,c,d,e] (for example, 5 features)
## let m be the number of features

inputs = [-2,1,2,0,1]
sigma = cov2corr(np.eye(5))  ## old empirical covariance matrix
n = 23  ## 23rd day since beginning
k = 2   ## second day of the week
ranks = []
for i in range(m):
  ## convert the raw input to onlinesort class
  new_obs = OnlineSort(inputs[i],23,2)
  new_info = gather_data(dict_data[str(i)],new_obs)
  dict_data[str(i)] = new_info["new_data"]
  outcome = split_update_eps(dict_eps[str(i)],new_info["new_eps"],new_info["delta"])
  dict_eps[str(i)] =  = outcome["new_sorted_eps"]
  ranks.append(outcome["rank"])
  
sigma = var_update(ranks,n,sigma)
stat =  test_stat(ranks,n,sigma)
## do percentile correction